# Data Setup and Preprocessing Pipelines (The Professional Way)

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/02_preprocessing_pipelines.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Audit data types and fix common pandas pitfalls (strings, categories, dates)
2. Handle missing values without leaking information
3. Build a preprocessing + model Pipeline with `ColumnTransformer`
4. Separate "fit on train only" logic from evaluation logic
5. Use Gemini to draft pipeline code and then harden it (tests + comments)

---

> **📋 Participation Reminder:** This notebook contains **2 PAUSE-AND-DO exercises**. You are expected to complete all exercises before submitting your notebook.

---

## 💼 Why This Matters: The Messy Reality of Real-World Data

You just joined **HomeValue Analytics**, a real estate tech startup that helps buyers and sellers make informed pricing decisions in California. Your CEO hands you a dataset of 20,000+ neighborhood-level housing records and says: *"Build me a model that predicts median house values."*

You open the data and discover: features on wildly different scales, potential missing values, and mixed numeric types. Before any prediction can happen, this data needs to be cleaned and transformed — reliably, repeatably, and without leaking information.

> **Today's focus:** Building reusable preprocessing pipelines that transform raw housing data into model-ready features.

---

## 1. Setup: Imports and Configuration

Before building any model, we load the libraries that power our workflow. Each import serves a specific role:

- **pandas / numpy** — data manipulation and numeric computation
- **matplotlib / seaborn** — visualization
- **sklearn.pipeline.Pipeline** — chains preprocessing and modeling into a single reproducible object
- **sklearn.compose.ColumnTransformer** — applies different transformations to different column groups
- **sklearn.preprocessing** — scaling (`StandardScaler`) and encoding (`OneHotEncoder`)
- **sklearn.impute.SimpleImputer** — fills missing values so downstream models receive clean inputs
- **sklearn.linear_model.LinearRegression** — our baseline regression model

We also lock `RANDOM_SEED = 474` so that every split and every model produces the same result each time you run the notebook.

In [11]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn scikit-learn --quiet

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import warnings

# Display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Set random seed for reproducibility
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)

print("✓ Setup complete!")
print(f"Random seed: {RANDOM_SEED}")

✓ Setup complete!
Random seed: 474


**Reading the output:**

The `✓ Setup complete!` confirmation tells you every import succeeded and the environment is ready. The most important line is `Random seed: 474` — this guarantees that all random operations (data splits, model initialization) are **deterministic**: you will get the exact same numbers every time you run the notebook.

**Key takeaway:** Always run the setup cell first. If any import fails (e.g., `ModuleNotFoundError`), uncomment the `!pip install` line at the top and rerun.

---

## 2. Load Dataset and Create Splits

We continue with the **California Housing** dataset introduced in the previous notebook. It contains **20,640 census block-groups** described by 8 numeric features (median income, house age, average rooms, etc.) with the target being **median house value** (in \$100k units). Because the data is pre-cleaned by scikit-learn, we can focus entirely on the preprocessing and pipeline mechanics.

As in every notebook in this course, we use a **60 / 20 / 20** train / validation / test split created in a single two-step call to `train_test_split`. Splitting *before* any transformation is the first line of defense against data leakage.

> 💡 **Gemini Prompt:** "Using scikit-learn, load the California Housing dataset as a pandas DataFrame (use `fetch_california_housing(as_frame=True)`). Print the dataset shape, list all column names, and display the first 5 rows."
>
> **After running, verify:**
> - Shape is `(20640, 9)` (20,640 rows, 8 features + 1 target)
> - Features include `MedInc`, `HouseAge`, `AveRooms`, `AveBedrms`, `Population`, `AveOccup`, `Latitude`, `Longitude`
> - Target column is `MedHouseVal` (median house value in \$100k units)


In [2]:
# Load California Housing dataset
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing(as_frame=True)
df = california.frame

print(f"Dataset shape: {df.shape}")
print(f"\nFeatures: {df.columns.tolist()}")
df.head()

Dataset shape: (20640, 9)

Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'MedHouseVal']


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.325,41.0,6.984,1.024,322.0,2.556,37.88,-122.23,4.526
1,8.301,21.0,6.238,0.972,2401.0,2.110,37.86,-122.22,3.585
2,7.257,52.0,8.288,1.073,496.0,2.802,37.85,-122.24,3.521
3,5.643,52.0,5.817,1.073,558.0,2.548,37.85,-122.25,3.413
4,3.846,52.0,6.282,1.081,565.0,2.181,37.85,-122.25,3.422


**Reading the output:**

The dataset has **20,640 rows** (one per census block-group) and **9 columns** — 8 features plus the target `MedHouseVal`. The `.head()` preview shows the first five rows with all columns visible. Key things to notice:

- **MedInc** (median income, in \$10k units) varies noticeably even in these five rows, hinting at its strong predictive power.
- **Latitude / Longitude** are geographic coordinates — they encode *location*, which heavily influences California house prices.
- **MedHouseVal** (target) is measured in \$100k units, so a value of 4.526 means a median price of \~\$452,600.
- All values are numeric floats — there are no string or categorical columns to worry about in this dataset.

**Why this matters:** Understanding the shape and units of your data before writing any pipeline code prevents surprises later (e.g., accidentally treating Latitude as a feature that needs one-hot encoding).

---

> 💡 **Gemini Prompt:** "Split the California Housing data into features (`X`) and target (`y`, column `MedHouseVal`). Then create a 60/20/20 train/validation/test split using `train_test_split` with `random_state=474`. Print the number of samples and percentage for each split."
>
> **After running, verify:**
> - Train set is \~12,384 samples (\~60%)
> - Validation set is \~4,128 samples (\~20%)
> - Test set is \~4,128 samples (\~20%)
> - Percentages sum to 100%

In [3]:
# Separate features and target
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

# Split: 60% train, 20% validation, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED
)

print("=== SPLIT SIZES ===")
print(f"Train: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation: {len(X_val)} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"Test: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")

=== SPLIT SIZES ===
Train: 12384 samples (60.0%)
Validation: 4128 samples (20.0%)
Test: 4128 samples (20.0%)


**Reading the output:**

The split produces three non-overlapping subsets:

- **Train:** \~12,384 samples (60.0 %) — used to fit model parameters.
- **Validation:** \~4,128 samples (20.0 %) — used to tune hyperparameters and compare models.
- **Test:** \~4,128 samples (20.0 %) — held out until the very end to estimate real-world performance.

The two-step `train_test_split` strategy (first carve off 20 % for test, then split the remaining 80 % into 75/25 to get 60/20) is the course standard. Using `random_state=RANDOM_SEED` (474) ensures these exact same rows land in each subset every time the notebook is run.

**Key takeaway:** All preprocessing decisions (imputer statistics, scaler means/stds) will be computed on the training set only. The validation and test sets exist solely for *evaluation* — never for fitting.

---

## 3. Data Audit Report Function

Before applying any transformation, a professional data scientist audits the training data for **data types**, **missing values**, and **cardinality** (unique-value counts). The function below wraps those checks into a single reusable report. Running it on every split lets you confirm that the train, validation, and test sets share the same schema — a simple but critical sanity check that catches silent errors early.

> 💡 **Gemini Prompt:** "Write a Python function `make_data_report(df, name)` that prints a formatted data audit report showing: the DataFrame shape, a table with each column's dtype, missing count, missing percentage, unique count, and unique percentage, plus a summary of total missing values and column type counts. Return the report as a DataFrame. Then run it on `X_train`."
>
> **After running, verify:**
> - Report shows all 8 feature columns (no target column)
> - All columns have dtype `float64`
> - Missing count is 0 for all columns (California Housing has no missing data)
> - Non-numeric columns count is 0


In [4]:
def make_data_report(df, name="Dataset"):
    """
    Generate a comprehensive data audit report.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The dataframe to audit
    name : str
        Name of the dataset for reporting
    
    Returns:
    --------
    pd.DataFrame
        Report with types, missingness, and unique counts
    """
    print(f"\n{'='*60}")
    print(f"DATA AUDIT REPORT: {name}")
    print(f"{'='*60}")
    
    print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Build report dataframe
    report = pd.DataFrame({
        'dtype': df.dtypes,
        'missing_count': df.isnull().sum(),
        'missing_pct': (df.isnull().sum() / len(df) * 100).round(2),
        'unique_count': df.nunique(),
        'unique_pct': (df.nunique() / len(df) * 100).round(2)
    })
    
    print("\n=== COLUMN AUDIT ===")
    print(report)
    
    # Summary statistics
    print(f"\n=== SUMMARY ===")
    print(f"Total missing values: {df.isnull().sum().sum():,}")
    print(f"Columns with missing data: {(df.isnull().sum() > 0).sum()}")
    print(f"Numeric columns: {df.select_dtypes(include=[np.number]).shape[1]}")
    print(f"Non-numeric columns: {df.select_dtypes(exclude=[np.number]).shape[1]}")
    
    return report

# Run audit on training data
train_report = make_data_report(X_train, "Training Set")


DATA AUDIT REPORT: Training Set

Shape: 12,384 rows × 8 columns

=== COLUMN AUDIT ===
              dtype  missing_count  missing_pct  unique_count  unique_pct
MedInc      float64              0          0.0          8707       70.31
HouseAge    float64              0          0.0            52        0.42
AveRooms    float64              0          0.0         11853       95.71
AveBedrms   float64              0          0.0          9475       76.51
Population  float64              0          0.0          3320       26.81
AveOccup    float64              0          0.0         11609       93.74
Latitude    float64              0          0.0           806        6.51
Longitude   float64              0          0.0           799        6.45

=== SUMMARY ===
Total missing values: 0
Columns with missing data: 0
Numeric columns: 8
Non-numeric columns: 0


**Reading the output:**

The audit report for the **Training Set** shows:

- **Shape:** \~12,384 rows × 8 columns (60 % of 20,640 — exactly the training portion from our 60/20/20 split).
- **Data types:** All 8 features are `float64` — no type conversions needed.
- **Missing values:** Zero across every column. In real-world projects, this row would rarely be all zeros, so having the check in place is important for the day you encounter incomplete data.
- **Unique counts:** Features like Latitude and Longitude have thousands of unique values (continuous), while HouseAge has only \~52 (discrete, capped at 52 years by the census).

**Why this matters:** Running this report *on the training set only* establishes the baseline data profile. When you compare it to the validation and test reports (next cell), any schema mismatch — different types, unexpected missingness, or wildly different unique counts — would signal a problem in how the splits were created.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Implement `make_data_report(df)` that returns types, missingness %, and unique counts.

**Instructions:**
1. The function is already implemented above
2. Run it on `X_train`, `X_val`, and `X_test`
3. Document any differences you observe between splits
4. Write your observations in the cell below

---

> 💡 **Gemini Prompt:** "Run the `make_data_report` function on the validation set (`X_val`) and test set (`X_test`) to audit both splits."
>
> **After running, verify:**
> - Both reports show the same 8 feature columns as the training report
> - Missing values are 0 across all columns in both splits
> - Row counts match the expected validation (\~4,128) and test (\~4,128) sizes

In [5]:
# Run audit on all splits
val_report = make_data_report(X_val, "Validation Set")
test_report = make_data_report(X_test, "Test Set")


DATA AUDIT REPORT: Validation Set

Shape: 4,128 rows × 8 columns

=== COLUMN AUDIT ===
              dtype  missing_count  missing_pct  unique_count  unique_pct
MedInc      float64              0          0.0          3417       82.78
HouseAge    float64              0          0.0            51        1.24
AveRooms    float64              0          0.0          4054       98.21
AveBedrms   float64              0          0.0          3642       88.23
Population  float64              0          0.0          2158       52.28
AveOccup    float64              0          0.0          4028       97.58
Latitude    float64              0          0.0           652       15.79
Longitude   float64              0          0.0           659       15.96

=== SUMMARY ===
Total missing values: 0
Columns with missing data: 0
Numeric columns: 8
Non-numeric columns: 0

DATA AUDIT REPORT: Test Set

Shape: 4,128 rows × 8 columns

=== COLUMN AUDIT ===
              dtype  missing_count  missing_pct  uni

### YOUR OBSERVATIONS HERE:

**Observation 1:**  
[Document any differences in data types across splits]

**Observation 2:**  
[Document any missingness patterns]

**Observation 3:**  
[Document any unique value differences]

---

## 4. Preprocessing Pipeline Template

### 4.1 Identify Column Types

> 💡 **Gemini Prompt:** "Using `select_dtypes`, identify which columns in `X_train` are numeric and which are categorical. Store them in lists called `numeric_features` and `categorical_features`. Print both lists with counts."
>
> **After running, verify:**
> - All 8 features are classified as numeric (`MedInc`, `HouseAge`, `AveRooms`, `AveBedrms`, `Population`, `AveOccup`, `Latitude`, `Longitude`)
> - Categorical features list is empty (California Housing is all-numeric)
> - Total count is 8 numeric + 0 categorical = 8 features


In [6]:
# Identify numeric and categorical columns
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

print("=== FEATURE TYPES ===")
print(f"\nNumeric features ({len(numeric_features)}):")
for feat in numeric_features:
    print(f"  - {feat}")

print(f"\nCategorical features ({len(categorical_features)}):")
if categorical_features:
    for feat in categorical_features:
        print(f"  - {feat}")
else:
    print("  (none)")

=== FEATURE TYPES ===

Numeric features (8):
  - MedInc
  - HouseAge
  - AveRooms
  - AveBedrms
  - Population
  - AveOccup
  - Latitude
  - Longitude

Categorical features (0):
  (none)


**Reading the output:**

The audit shows **8 numeric features** (MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude) and **0 categorical features**. This means every column in `X_train` is already numeric (`float64`).

We use `select_dtypes(include=[np.number])` rather than hard-coding column names because it is **portable**: if a future dataset contains `object`-type columns (e.g., neighborhood names, zip codes), the same two lines will automatically route them into the categorical list without any code changes.

**Why this matters:** Knowing the column types determines which preprocessing steps each group needs — numeric features get imputation and scaling, categorical features get imputation and one-hot encoding. This column-type split is the foundation of the `ColumnTransformer` we build next.

---

### 4.2 Build Preprocessing Pipeline

**Pipeline Design Principles:**
1. Fit transformers ONLY on training data
2. Apply the same transformation to validation and test
3. Handle numeric and categorical features separately
4. Chain all steps together to prevent leakage

### Understanding Each Transformer Step

Before we write the code, let's understand **why** each component exists and what problem it solves.

**Numeric Transformer** (`SimpleImputer` → `StandardScaler`):

| Step | What it does | Why we need it |
|------|-------------|----------------|
| `SimpleImputer(strategy='median')` | Fills every `NaN` in a numeric column with that column's **median**, computed from the training set only | Scikit-learn models **crash on `NaN`** — even a single missing value causes an error.<br><br>We choose **median** over mean because median is robust to outliers: a few mansions with 50 rooms won't distort the fill value the way they would distort the mean. |
| `StandardScaler()` | Rescales each feature to **mean ≈ 0, std ≈ 1** | Features live on wildly different scales (MedInc: 0–15 vs. Population: 0–35,000).<br><br>Without scaling: (1) regularized models like Ridge and Lasso unfairly penalize large-scale features, (2) gradient-based optimizers converge slowly, and (3) distance-based models like KNN are dominated by the largest-scale feature. |

**Categorical Transformer** (`SimpleImputer` → `OneHotEncoder`):

| Step | What it does | Why we need it |
|------|-------------|----------------|
| `SimpleImputer(strategy='constant', fill_value='missing')` | Fills every `NaN` in a text/categorical column with the literal string `"missing"` | We can't compute a median for text — there is no "middle" word.<br><br>Instead, we create an explicit `"missing"` category so the encoder has a valid string to work with.<br>Alternative: `strategy='most_frequent'` fills with the most common category. |
| `OneHotEncoder(handle_unknown='ignore', sparse_output=False)` | Converts each category into a **separate binary (0/1) column** | Models need numbers, not strings.<br><br>E.g., a `Color` column with values [Red, Blue, Green] becomes three columns: `Color_Red`, `Color_Blue`, `Color_Green`, each containing 0 or 1.<br><br>The flag `handle_unknown='ignore'` means if the validation/test set contains a category never seen during training, it gets all-zero columns instead of crashing. |

> **💡 "But this dataset has no missing values and no categorical columns — why bother?"**  
> Because a production-grade pipeline must handle messy data gracefully. The imputer is a **no-op** today (nothing to fill), and the categorical transformer is **idle** (no text columns to encode). But when you later work with a real-world dataset that has gaps and string features, this same pipeline structure works without any code changes. Design for the general case; benefit today from the clean case.

> 💡 **Gemini Prompt:** "Build a scikit-learn preprocessing pipeline using `ColumnTransformer`. For numeric features, create a sub-pipeline that applies `SimpleImputer(strategy='median')` then `StandardScaler()`. For categorical features, create a sub-pipeline with `SimpleImputer(strategy='constant', fill_value='missing')` then `OneHotEncoder(handle_unknown='ignore', sparse_output=False)`. Print a confirmation with the feature counts."
>
> **After running, verify:**
> - Preprocessor is a `ColumnTransformer` with two transformers: `num` and `cat`
> - Numeric features count is 8
> - Categorical features count is 0
> - No errors about missing column names


In [7]:
# Numeric preprocessing: impute + scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing: impute + encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Drop any columns not specified
)

print("✓ Preprocessor pipeline created!")
print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

✓ Preprocessor pipeline created!

Numeric features: 8
Categorical features: 0


**Reading the output:**

The confirmation message tells us the `ColumnTransformer` is ready with **8 numeric features** and **0 categorical features**. Under the hood, three data-flow paths are defined:

1. **`num` path** (numeric columns) → `SimpleImputer(strategy='median')` → `StandardScaler()`. Median imputation is robust to outliers (recall AveRooms and AveOccup have extreme values). Standard scaling centers each feature to mean 0 and std 1, which benefits regularized models introduced in upcoming notebooks.
2. **`cat` path** (categorical columns) → `SimpleImputer(strategy='constant')` → `OneHotEncoder()`. This path is currently idle because all features are numeric, but including it makes the pipeline **portable**: if you swap in a dataset with categorical columns, the same code works without changes.
3. **`remainder='drop'`** — any column not explicitly listed is discarded. This is a safety net that prevents accidental target leakage.

**Key takeaway:** Defining the transformer *before* seeing results forces you to think about each column group's needs up front — a habit that prevents ad-hoc, error-prone transformations later.

---

### 4.2.1 Beyond This Dataset: Handling Strings and Dates

The California Housing dataset is all-numeric, but real-world datasets often contain **text strings** and **date/time columns**. Here is how each type fits into the `ColumnTransformer` pattern we just built.

**String / Text Columns:**

| Scenario | Recommended Tool | Example Columns |
|----------|-----------------|-----------------|
| Few unique values (< \~20) | `OneHotEncoder` (already in our `cat` pipeline) | Product color, U.S. state, payment method |
| Ordered categories with a natural ranking | `OrdinalEncoder(categories=[['Low','Medium','High']])` | Education level, satisfaction rating, T-shirt size |
| Free-form text (sentences, paragraphs) | `CountVectorizer` or `TfidfVectorizer` | Customer reviews, email subject lines |
| Very high cardinality (1,000+ unique values) | `TargetEncoder` or frequency-based encoding | City name, product SKU, zip code |

> **💡 Key distinction:** `OneHotEncoder` creates one column per category — great when categories are few, but it explodes the feature count when there are thousands of unique values. For high-cardinality columns, `TargetEncoder` (available in scikit-learn ≥ 1.3) replaces each category with the mean of the target for that category, producing just **one** numeric column.

**Date / Datetime Columns:**

Dates are not directly usable by models — "2027-06-01" is a string, not a number. The standard approach is to **extract numeric features** from them before the data enters the pipeline:

```python
# Parse the date column (do this BEFORE building the pipeline)
df['date'] = pd.to_datetime(df['date'])

# Extract useful numeric features
df['year']        = df['date'].dt.year
df['month']       = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek   # 0=Monday, 6=Sunday
df['is_weekend']  = (df['date'].dt.dayofweek >= 5).astype(int)
df['quarter']     = df['date'].dt.quarter

# Drop the original date column — the pipeline can't process it
df = df.drop(columns=['date'])
```

Once extracted, these columns are **plain numbers** and flow through the numeric transformer (imputer + scaler) like any other feature.

> **💡 Rule of thumb:** Convert dates to numbers *before* building the pipeline, then drop the original date column. The extracted features (month, day_of_week, is_weekend, etc.) carry the temporal signal that the model can learn from; the raw datetime string does not.

---

### 4.3 Full Pipeline: Preprocessing + Model

Scikit-learn's `Pipeline` chains the `ColumnTransformer` (preprocessing) with a `LinearRegression` (model) into **one callable object**. This design has three major benefits:

1. **No leakage** — `pipeline.fit(X_train, y_train)` fits the scaler statistics *only* on training data; `pipeline.score(X_val, y_val)` transforms the validation set with those same statistics before predicting.
2. **Deployment-ready** — you can serialize the pipeline with `joblib` and run `pipeline.predict(new_data)` in production without rewriting any preprocessing code.
3. **Reproducibility** — the exact sequence of steps is recorded in the pipeline object itself, eliminating copy-paste errors between notebooks or scripts.

> 💡 **Gemini Prompt:** "Create a full scikit-learn `Pipeline` that chains the `preprocessor` (from the previous step) with a `LinearRegression` model. Print the pipeline structure, fit it on `X_train`/`y_train`, then compute and print the R-squared scores on both training and validation sets, along with their difference."
>
> **After running, verify:**
> - Pipeline has two steps: `preprocessor` and `regressor`
> - Train R-squared is approximately 0.61
> - Validation R-squared is approximately 0.59–0.61
> - The difference (train minus validation) is small (< 0.03), indicating no severe overfitting


In [8]:
# Create full pipeline: preprocessing + model
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

print("=== FULL PIPELINE ===")
print(full_pipeline)

# Fit on training data ONLY
print("\nFitting pipeline on training data...")
full_pipeline.fit(X_train, y_train)
print("✓ Pipeline fitted!")

# Evaluate on validation set
train_score = full_pipeline.score(X_train, y_train)
val_score = full_pipeline.score(X_val, y_val)

print(f"\n=== SCORES ===")
print(f"Train R²: {train_score:.4f}")
print(f"Validation R²: {val_score:.4f}")
print(f"Difference: {train_score - val_score:.4f}")

=== FULL PIPELINE ===
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MedInc', 'HouseAge',
                                                   'AveRooms', 'AveBedrms',
                                                   'Population', 'AveOccup',
                                                   'Latitude', 'Longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
 

**Reading the output:**

The printed `Pipeline` object shows the two-step chain: first the `ColumnTransformer` (preprocessing), then `LinearRegression` (model). After fitting on the training set only, we obtain:

- **Train R² ≈ 0.6035** — the model explains about 60 % of the variance in median house values on the data it was trained on.
- **Validation R² ≈ 0.6169** — performance on unseen validation data is essentially the same (even slightly higher due to random-split variation).
- **Difference ≈ −0.01** — the tiny gap (and the fact that validation is not *lower*) tells us there is **no overfitting**. The model is equally mediocre on both sets, which is the hallmark of a high-bias, low-variance linear model.

**Why this matters:** An R² of \~0.60 is a reasonable *baseline*. It means that a simple linear combination of the 8 features already captures the dominant income-drives-house-value relationship. In the next notebooks, we will try feature engineering and more flexible models (trees, ensembles) to push this number higher — but this pipeline ensures we always have a disciplined, leak-free starting point.

---

### 4.4 Inspecting the Pipeline

Understanding what features are created:

> 💡 **Gemini Prompt:** "Extract the transformed feature names from the fitted pipeline using `get_feature_names_out()` on the preprocessor step. Print the total number of transformed features and list the first 10 names."
>
> **After running, verify:**
> - Total transformed features equals 8 (all numeric, no one-hot expansion needed)
> - Feature names are prefixed with `num__` (e.g., `num__MedInc`, `num__HouseAge`)
> - All 8 original numeric feature names appear in the output


In [9]:
# Get feature names after transformation
try:
    feature_names = full_pipeline.named_steps['preprocessor'].get_feature_names_out()
    print(f"=== TRANSFORMED FEATURES ===")
    print(f"Total features after preprocessing: {len(feature_names)}")
    print(f"\nFirst 10 features:")
    for i, name in enumerate(feature_names[:10], 1):
        print(f"  {i}. {name}")
    
    if len(feature_names) > 10:
        print(f"  ... and {len(feature_names) - 10} more")
except:
    print("Feature names not available (older sklearn version)")
    print(f"Estimated features: {len(numeric_features)} numeric features (scaled)")

=== TRANSFORMED FEATURES ===
Total features after preprocessing: 8

First 10 features:
  1. num__MedInc
  2. num__HouseAge
  3. num__AveRooms
  4. num__AveBedrms
  5. num__Population
  6. num__AveOccup
  7. num__Latitude
  8. num__Longitude


**Reading the output:**

The `get_feature_names_out()` method lists every column that exits the `ColumnTransformer`. Each name carries a **prefix** that traces it back to the transformer that created it:

- `num__MedInc`, `num__HouseAge`, … — the 8 original numeric features, now standardized (mean ≈ 0, std ≈ 1) by the `num` pipeline.

Since the dataset has **zero categorical columns**, only the `num` transformer fires; the `cat` transformer produces no output columns, and `remainder='drop'` discards nothing because every column is already routed. The result is **8 features in → 8 features out**, just rescaled.

**Why this matters:** Being able to map transformed columns back to their original names is essential for model interpretation. When you inspect linear-regression coefficients or feature-importance scores in upcoming notebooks, these prefixed names will tell you exactly which feature each value belongs to.

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Create a full sklearn Pipeline and run one validation score.

**Instructions:**
1. The pipeline is already created above
2. Modify the pipeline to try different imputation strategies:
   - Change median to mean for numeric features
   - Try different imputation for categorical features
3. Compare the validation scores
4. Document your findings below

---

> 💡 **Gemini Prompt:** "Create an alternative preprocessing pipeline that uses `SimpleImputer(strategy='mean')` instead of median for numeric features, keeping everything else the same. Fit it with `LinearRegression`, evaluate on train and validation sets, and print a comparison table showing both pipelines' R-squared scores side by side."
>
> **After running, verify:**
> - Both pipelines produce similar R-squared values (California Housing has no missing data, so imputation strategy has minimal impact)
> - The difference in validation scores between mean and median imputation is very small (< 0.001)
> - Train scores are slightly higher than validation scores for both pipelines


In [10]:
# YOUR CODE HERE: Experiment with different preprocessing strategies

# Example: Try mean imputation instead of median
numeric_transformer_v2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Changed from median
    ('scaler', StandardScaler())
])

preprocessor_v2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_v2, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

full_pipeline_v2 = Pipeline(steps=[
    ('preprocessor', preprocessor_v2),
    ('regressor', LinearRegression())
])

# Fit and evaluate
full_pipeline_v2.fit(X_train, y_train)
train_score_v2 = full_pipeline_v2.score(X_train, y_train)
val_score_v2 = full_pipeline_v2.score(X_val, y_val)

print("=== COMPARISON ===")
print(f"\nOriginal (median imputation):")
print(f"  Train R²: {train_score:.4f}")
print(f"  Val R²: {val_score:.4f}")

print(f"\nAlternative (mean imputation):")
print(f"  Train R²: {train_score_v2:.4f}")
print(f"  Val R²: {val_score_v2:.4f}")

print(f"\nDifference in validation score: {val_score_v2 - val_score:.6f}")

=== COMPARISON ===

Original (median imputation):
  Train R²: 0.6035
  Val R²: 0.6169

Alternative (mean imputation):
  Train R²: 0.6035
  Val R²: 0.6169

Difference in validation score: 0.000000


**Reading the output:**

The comparison table shows that the **median-imputation** and **mean-imputation** pipelines produce virtually identical R² scores on both the training and validation sets. The difference in validation R² is on the order of **10⁻⁶** — essentially zero.

This is expected: the California Housing dataset has **zero missing values**, so `SimpleImputer` is a no-op regardless of strategy. Neither median nor mean imputation actually changes any value.

**Key takeaway:** The imputer is included in the pipeline not because this dataset needs it, but because a *production-grade* pipeline must handle missing data gracefully when new, unseen records arrive. Designing the pipeline once — even if the imputer does nothing today — saves you from rewriting code the moment real-world messiness appears.

---

### YOUR FINDINGS HERE:

**Finding 1:**  
[Compare the two imputation strategies]

**Finding 2:**  
[Discuss which performs better and why]

**Finding 3:**  
[Any other observations about the pipeline]

---

## 5. Gemini Prompt Cards for Pipeline Generation

### Example Prompts:

**Prompt 1: Generate Pipeline**
```
Create a scikit-learn pipeline that:
1. Imputes missing values (median for numeric, most_frequent for categorical)
2. Scales numeric features using StandardScaler
3. Encodes categorical features using OneHotEncoder
4. Fits a LinearRegression model

Use ColumnTransformer to handle different feature types separately.
```

**Prompt 2: Debug Pipeline**
```
I'm getting an error when fitting my pipeline: [error message]
My pipeline code is: [code]
Help me debug this issue and explain what's wrong.
```

**Prompt 3: Extend Pipeline**
```
I have a working pipeline with imputation and scaling.
How can I add polynomial features (degree=2) only to numeric features
while keeping the categorical encoding unchanged?
```

**Remember:** After using Gemini, always:
1. Verify the code runs without errors
2. Check the output makes sense
3. Add your own comments explaining each step

## 6. Pipeline Done Right Checklist

Before moving on, verify your pipeline meets these standards:

### ✓ Pre-fitting Checks:
- [ ] Split data BEFORE building pipeline
- [ ] Identified all numeric and categorical features
- [ ] Handled missing values with appropriate strategy
- [ ] Separate transformers for different feature types

### ✓ Fitting Checks:
- [ ] Pipeline fitted ONLY on training data
- [ ] No data leakage from validation or test sets
- [ ] All preprocessing steps are inside the pipeline
- [ ] Model is the last step in the pipeline

### ✓ Evaluation Checks:
- [ ] Evaluated on validation set (not test set)
- [ ] Training and validation scores are reasonable
- [ ] No major signs of overfitting (huge train/val gap)
- [ ] Pipeline can transform new data without refitting

### ✓ Code Quality:
- [ ] All parameters are explicit (no hidden defaults)
- [ ] Steps are named clearly
- [ ] Comments explain "why" not just "what"
- [ ] Can explain every step if asked

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Data Auditing**: Systematic checks for types, missingness, and unique values
2. **Preprocessing Patterns**: Separate handling for numeric vs categorical features
3. **Pipeline Architecture**: ColumnTransformer + Pipeline prevents leakage
4. **Fit/Transform Discipline**: Fit on train, transform on train/val/test
5. **Gemini Integration**: Use AI to draft, then verify and document

### Critical Rules:

> **"Fit ONLY on training data"**  
> Any statistics (mean, median, categories) must be computed from training data only.

> **"Pipeline wraps everything"**  
> If you do it manually, you risk leakage. Put it in the pipeline.

### Next Steps:

- The next notebook will cover regression metrics and baseline models
- We'll use today's pipeline structure for all future models
- Start thinking about your project dataset

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## Bibliography

- scikit-learn User Guide: [Pipelines and composite estimators](https://scikit-learn.org/stable/modules/compose.html)
- scikit-learn User Guide: [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
- scikit-learn User Guide: [Preprocessing data](https://scikit-learn.org/stable/modules/preprocessing.html)
- Pedregosa et al. (2011). "Scikit-learn: Machine Learning in Python." *Journal of Machine Learning Research*, 12, 2825-2830.
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* (ISLP). Springer.

---



<center>

Thank you!

</center>